In [1]:
import numpy as np
import h5py

from ase.build import bulk
from ase.calculators.emt import EMT
from ase.io import read
from hilde.helpers.supercell import make_cubic_supercell
from hilde.tasks.calculate import calculate_multiple
from hilde.structure.io import inform

from hilde.phono3py import wrapper as ph3

/home/purcell/bin/python/anaconda3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
atoms = bulk('Al', cubic=True)

In [17]:
sc2, smatrix2 = make_cubic_supercell(atoms, 32)
sc3, smatrix3 = make_cubic_supercell(atoms, 32)
inform(sc3)


Geometry info for:
  input geometry:    Al32
  Symmetry prec.:    1e-05
  Number of atoms:   32
  Species:           Al
  Periodicity:       [ True  True  True]
  Lattice:  
    [8.1 0.  0. ]
    [0.  8.1 0. ]
    [0.  0.  8.1]

  Spacegroup:          Fm-3m (225)
  Wyckoff positions:   32*a
  Equivalent atoms:    32*a
  Standard lattice:  
    [4.05 0.   0.  ]
    [0.   4.05 0.  ]
    [0.   0.   4.05]
  supercell_matrix: [2, 0, 0, 0, 2, 0, 0, 0, 2]

Cell lengths and angles [Å, °]:
  a, b, c:      8.1000      8.1000      8.1000
  α, β, γ:     90.0000     90.0000     90.0000
  Volume:     531.4410 Å**3


In [18]:
phono3py_settings = {
    'atoms': atoms,
    'fc3_supercell_matrix': smatrix3,
    'cutoff_pair_distance': 3
}

In [19]:
phonon3, sc2, sc3, scs2, scs3 = ph3.preprocess(**phono3py_settings)

In [20]:
print(f'Number of FC2 displacements: {len(scs2)}')
print(f'Number of FC3 displacements: {len(scs3)}')
print(f'Number of FC3 displacements that were created: {len([sc for sc in scs3 if sc])}')

Number of FC2 displacements: 1
Number of FC3 displacements: 37
Number of FC3 displacements that were created: 15


In [21]:
calc = EMT()

## Compute Forces

In [22]:
scs2_computed = calculate_multiple(scs2, calc, 'test_run/fc2')
fc2_forces = ph3.get_forces(scs2_computed)

In [23]:
%time scs3_computed = calculate_multiple(scs3, calc, 'test_run/fc3')
fc3_forces = ph3.get_forces(scs3_computed)
len(scs3_computed), len(scs3)

CPU times: user 729 ms, sys: 8 ms, total: 737 ms
Wall time: 741 ms


(37, 37)

In [24]:
[s for s in scs3_computed if s][-1].info

{'displacement_id': 29}

## Produce Force Constants

In [25]:
from hilde.structure.convert import to_Atoms, to_phonopy_atoms
from hilde import konstanten as const
from phono3py.phonon3 import Phono3py

ph_atoms = to_phonopy_atoms(atoms, wrap=True)
phonon3 = Phono3py(
    ph_atoms,
    supercell_matrix=smatrix3,
    mesh=[11,11,11],
    symprec=0.0001,
    is_symmetry=True,
    symmetrize_fc3q=False,
    frequency_factor_to_THz=const.omega_to_THz,
    log_level=2,
)
phonon3.generate_displacements(
    distance=0.03, cutoff_pair_distance=3.0
)

In [26]:
phonon3.produce_fc2(fc2_forces)
print(phonon3._displacement_dataset)

{'natom': 32, 'cutoff_distance': 3.0, 'first_atoms': [{'number': 0, 'direction': [1, 0, 0], 'displacement': array([0.03, 0.  , 0.  ]), 'second_atoms': [{'number': 0, 'direction': array([1, 1, 0]), 'displacement': array([0.0212132, 0.0212132, 0.       ]), 'pair_distance': 0.0, 'included': True}, {'number': 0, 'direction': array([-1, -1,  0]), 'displacement': array([-0.0212132, -0.0212132,  0.       ]), 'pair_distance': 0.0, 'included': True}, {'number': 1, 'direction': array([1, 1, 0]), 'displacement': array([0.0212132, 0.0212132, 0.       ]), 'pair_distance': 4.05, 'included': False}, {'number': 1, 'direction': array([-1, -1,  0]), 'displacement': array([-0.0212132, -0.0212132,  0.       ]), 'pair_distance': 4.05, 'included': False}, {'number': 2, 'direction': array([1, 1, 1]), 'displacement': array([0.01732051, 0.01732051, 0.01732051]), 'pair_distance': 4.05, 'included': False}, {'number': 2, 'direction': array([-1, -1, -1]), 'displacement': array([-0.01732051, -0.01732051, -0.0173205

In [27]:
phonon3.produce_fc3(fc3_forces)
print(phonon3._displacement_dataset)

Computing fc3[ 1, x, x ] using numpy.linalg.pinv with a displacement:
    [ 0.0300  0.0000  0.0000]
Expanding fc3
Cutting-off fc3 (cut-off distance: 3.000000)
Building atom mapping table...
Creating contracted fc3...
{'natom': 32, 'cutoff_distance': 3.0, 'first_atoms': [{'number': 0, 'direction': [1, 0, 0], 'displacement': array([0.03, 0.  , 0.  ]), 'second_atoms': [{'number': 0, 'direction': array([1, 1, 0]), 'displacement': array([0.0212132, 0.0212132, 0.       ]), 'pair_distance': 0.0, 'included': True, 'delta_forces': array([[-6.82972514e-02, -6.83263237e-02, -4.60395611e-15],
       [-3.31860483e-03,  6.23059147e-04, -2.66800471e-15],
       [ 6.21612152e-04, -3.32245496e-03,  1.38430933e-15],
       [-4.21990194e-06, -4.51137653e-06, -2.42861287e-17],
       [ 6.20524588e-04,  6.20439109e-04,  5.55111512e-16],
       [-1.02236605e-06, -2.68732906e-07,  3.54924423e-15],
       [-3.17695756e-07,  2.83395779e-06, -2.22044605e-15],
       [ 5.25701334e-07,  5.29406784e-07,  1.4206936

In [ ]:
fc2 = phonon3.get_fc2()
fc3 = phonon3.get_fc3()

In [ ]:
q_mesh = 3*[3]
phonon3_new = ph3.prepare_phono3py(**phono3py_settings,
                                   fc2=fc2,
                                   fc3=fc3,
                                   q_mesh=q_mesh,
                                   log_level=0)

In [ ]:
phonon3_new.run_thermal_conductivity(write_kappa=True)

In [ ]:
with h5py.File('kappa-m{}.hdf5'.format(''.join(map(str, q_mesh)))) as kappa:
    print(kappa['frequency'][:])
    print(kappa['kappa'][:][np.where(kappa['temperature'][:] == 300)][:3])
    # assert 5 < kappa['frequency'][:].max() < 10, kappa['frequency'][:].max()
    # assert 25 < kappa['kappa'][:][30][:3].mean() < 35